# RAG Based on HyDE
- Hybrid Search
- ReRanker
- HyDE (Hypothetical Document Embeddings)

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [7]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f4322412da0>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f43602d68f0>])

### Embedding 모델 선택

In [8]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [9]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    
    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [10]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [11]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [12]:
depoly = False

In [13]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'BAAI/bge-reranker-large',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

## 4. Invocation (prediction)

In [14]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f422c615390>


In [15]:
#endpoint_name = "huggingface-pytorch-inference-2023-11-15-04-37-45-120" # ml.m5.2xlarge
endpoint_name = "huggingface-pytorch-inference-2023-11-15-07-53-21-605" # ml.g5.xlarge
deserializer = "application/json"

In [16]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [17]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.999653697013855}, {'label': 'LABEL_0', 'score': 0.025255175307393074}]
CPU times: user 20.1 ms, sys: 1.34 ms, total: 21.4 ms
Wall time: 79.3 ms


## 5. LangChainmOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [18]:
from utils.proc_docs import get_parameter

In [19]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_client = ssm,
    parameter_name = 'knox_opensearch_password',
)

In [20]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [21]:
index_name = "genai-poc-knox-incremental-1024c-256o-v7"

### OpenSearch Client 생성

In [22]:
from utils.opensearch import opensearch_utils

In [23]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 6. Retriever based on Hybrid Search + HyDE + ReRanker
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- Hybrid-Search에 대한 자세한 내용는 **"01_rag_hybrid_search.ipyno"** 에서 확인 가능합니다.
- [HyDE](https://medium.com/prompt-engineering/hyde-revolutionising-search-with-hypothetical-document-embeddings-3474df795af8)
![HyDE.png](../../../imgs/HyDE.png)

In [24]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [26]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE
    llm_emb=llm_emb,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [27]:
from utils.rag import show_context_used

In [28]:
query = "what is verify DM"

In [29]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

 Here is a concise passage answering what verify DM is:

Verify DM is a process used on social media platforms like Twitter to verify the authenticity of accounts belonging to public figures, celebrities, brands or organizations. When an account gets verified, it displays a blue verified badge icon next to the account name to indicate that the account is genuine and not an impersonator. The verification provides a level of authenticity and trust for high-profile accounts. Twitter and other platforms have a verification process that reviews accounts to ensure they meet criteria like being notable, active, and that there is a public interest in knowing an account is authentic.===== HyDE Answers =====
['what is verify DM', ' Here is a concise passage answering what verify DM is:\n\nVerify DM is a process used on social media platforms like Twitter to verify the authenticity of accounts belonging to public figures, celebrities, brands or organizations. When an account gets verified, it dis

- update parameters

In [30]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=30,
    #filter=[],
    filter=[
        {"term": {"metadata.project": "KS"}},
    ],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    verbose=False
)

In [31]:
query = "how to vefify DM"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

 Unfortunately I do not have enough context to generate a complete passage answering the question "how to vefify DM". A few points I could include if given more details:

- Specify what "DM" stands for (Direct Message, Dungeon Master, etc) and the context where one needs to verify it. 

- Provide steps to verify the DM if it refers to something like a Direct Message on a social media platform or messaging app. This might involve checking the sender's profile/identity, looking for verification badges, or trying to validate the content of the message.

- If DM refers to a Dungeon Master in a gaming context, provide suggestions on how a group could verify the qualifications/trustworthiness of someone wanting to be the DM for their group. This could include checking references, observing their DMing style in a one-shot game first, or looking at user reviews if they DM publicly.

Without more context I unfortunately can't write a full detailed passage to effectively answer "how to vefify DM

## 5. RAG using RetrievalQA powered by LangChain

In [33]:
from utils.rag import prompt_repo
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [34]:
PROMPT = prompt_repo.get_qa(prompt_type="answer_with_ref") # ["answer_only", "answer_with_ref"]
pprint (PROMPT.template)

('\n'
 '            \n'
 '\n'
 'Human:\n'
 '            You are a mater answer bot designed to answer software '
 "developer's questions.\n"
 "            I'm going to give you a context. Read the context carefully, "
 "because I'm going to ask you a question about it.\n"
 '\n'
 '            Here is the context: <context>{context}</context>\n'
 '\n'
 '            First, find the paragraphs or sentences from the context that '
 'are most relevant to answering the question, and then print them in numbered '
 'order.\n'
 '            The format of paragraphs or sentences to the question should '
 "look like what's shown between the <references></references> tags.\n"
 '            Make sure to follow the formatting and spacing exactly.\n'
 '\n'
 '            <references>\n'
 '            [Examples of question + answer pairs using parts of the given '
 'context, with answers written exactly like how Claude’s output should be '
 'structured]\n'
 '            </references>\n'
 '\n'
 '        

### Update Search Params (Optional)

In [35]:
from langchain.chains import RetrievalQA

In [44]:
opensearch_hybrid_retriever.update_search_params(
    k=3,
    minimum_should_match=0,
    filter=[],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    hyde=True, # enable hyde
    hyde_query=["web_search"], # query type in hyde 
    llm_text=llm_text, # llm for query augmentation in rag_fusion
    verbose=True
)

### Request

In [45]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [46]:
query = "How does RKP protect critical kernel data structures from modification?"
response = qa(query)

 Here is a draft passage answering the question "How does RKP protect critical kernel data structures from modification?":

RKP (Real-time Kernel Protection) is a security module that protects critical kernel data structures from unauthorized modification on Android devices. It works by locking down kernel data structures in read-only memory, preventing even processes with root access from directly overwriting things like process credentials or ID mappings. RKP also uses integrity checking to monitor changes to protected data structures during approved operations like process creation/deletion. Any discrepancies are detected and blocked. This allows proper functioning of the system while still sealing off sensitive areas like the process tree from potential malware or rogue apps. The multi-layered approach aims to secure the kernel without a significant loss of performance or compatibility.===== HyDE Answers =====
['How does RKP protect critical kernel data structures from modification

In [47]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])

##################################
query:  How does RKP protect critical kernel data structures from modification?
##################################


### Answer ###
 <references>
1. RKP protections are grouped into three areas: Kernel data - RKP"prevents modification of critical
kernel data structures.
</references>

<answer>
RKP prevents modification of critical kernel data structures.
</answer>


### Contexts ###
-----------------------------------------------
1. Chunk: 885 Characters
-----------------------------------------------
. This unique ability enables RKP to detect and prevent the most common kernel attacks. RKP
protections are grouped into three areas: Kernel code - RKP"prevents modification of kernel code and
logic. Kernel data - RKP"prevents modification of critical kernel data structures. Kernel control
flow - RKP"prevents Return-Oriented Programming (ROP) and Jump-Oriented Programming (JOP) attacks
that reuse existing kernel logic to piece together exploits from the 